In [1]:
import pinocchio as pin
from pinocchio import casadi as cpin
import casadi
import numpy as np
import example_robot_data as robex
#import matplotlib.pyplot as plt; plt.ion()
from pinocchio.visualize import GepettoVisualizer
from utils.meshcat_viewer_wrapper import  MeshcatVisualizer, colors

In [2]:
robot = robex.load('talos')
model = robot.model
cmodel = cpin.Model(robot.model)
data = model.createData()
cdata = cmodel.createData()

In [3]:
viz = MeshcatVisualizer(robot)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [4]:
viz.display(robot.q0)

In [5]:
q0 = robot.q0
v0 = np.zeros(robot.nv)
a0 = np.zeros(robot.nv)
x0 = np.concatenate([q0, v0])

In [6]:
nq = cmodel.nq
nv = cmodel.nv
nx = nq + nv
ndx = nv *2

cx = casadi.SX.sym('cx', nx, 1)
cdx = casadi.SX.sym('cx', ndx, 1)
ca = casadi.SX.sym('ca', nv, 1)
xref = casadi.SX.sym('cx', nx, 1)

IDX_LF = cmodel.getFrameId('leg_left_6_link')
IDX_RF = cmodel.getFrameId('leg_right_6_link')
IDX_LG = cmodel.getFrameId('gripper_left_base_link')
IDX_RG = cmodel.getFrameId('gripper_right_base_link')

integrate = casadi.Function('integrate', [cx, cdx], [ casadi.vertcat(cpin.integrate(cmodel,cx[:nq],cdx[:nv]),
                                                        cx[-nv:]+cdx[-nv:]) ] )
                                                        
com_position = casadi.Function('com', [cx], [cpin.centerOfMass(cmodel, cdata, cx[: nq])] )

cpin.forwardKinematics(cmodel, cdata, cx[: nq], cx[nq :], ca)
cpin.updateFramePlacements(cmodel, cdata)

lf_position = casadi.Function('lf_pos', [cx], [cdata.oMf[IDX_LF].translation])
lf_rotation = casadi.Function('lf_rot', [cx], [cdata.oMf[IDX_LF].rotation])
rf_position = casadi.Function('rf_pos', [cx], [cdata.oMf[IDX_RF].translation])
rf_rotation = casadi.Function('rf_rot', [cx], [cdata.oMf[IDX_RF].rotation])

lg_position = casadi.Function('lg_pos', [cx], [cdata.oMf[IDX_LG].translation])
lg_rotation = casadi.Function('lg_rot', [cx], [cdata.oMf[IDX_LG].rotation])
rg_position = casadi.Function('rg_pos', [cx], [cdata.oMf[IDX_RG].translation])
rg_rotation = casadi.Function('rg_rot', [cx], [cdata.oMf[IDX_RG].rotation])


# Tried with log method, which probably is more robust, but I was not able to make it work
#get_T_rf = casadi.Function('T_rf', [cx], [cpin.log6(cpin.SE3(rf_rotation(cx), rf_position(cx) ) ).vector])


# arm_distance = casadi.Function('arm_dist', [cx], [cpin.log6(cpin.SE3(lg_rotation(cx), lg_position(cx) ).inverse() *
 #                                                       cpin.SE3(rg_rotation(cx), rg_position(cx) )).vector])


In [9]:
opti = casadi.Opti()

dxs = opti.variable(ndx)
xs = integrate(x0, dxs)

cost = casadi.sumsqr(xs - x0)  #+ casadi.sumsqr(arm_distance(xs))

opti.minimize(cost)

# COM
opti.subject_to(opti.bounded(-0.08, com_position(xs)[0], 0.08))
opti.subject_to(opti.bounded(-0.02, com_position(xs)[1], 0.02))
opti.subject_to(opti.bounded(0.7, com_position(xs)[2], 0.9))

# Standing foot
opti.subject_to(rf_position(xs) - rf_position(x0) == 0)
opti.subject_to(rf_rotation(xs) - rf_rotation(x0) == 0)

# Free foot
opti.subject_to(lf_position(xs)[2] >= 0.4)
opti.subject_to(opti.bounded(0.05, lf_position(xs)[0:2], 0.1))

r_ref = pin.utils.rotate('z', 3.14 / 2) @ pin.utils.rotate('y', 3.14 / 2) # orientation target
opti.subject_to(opti.bounded(-0.0, lf_rotation(xs) - r_ref, 0.0))


In [10]:
opti.solver('ipopt')
opti.set_initial(dxs, np.zeros(ndx))
opti.solve()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.11.9, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      186
Number of nonzeros in inequality constraint Jacobian.:      144
Number of nonzeros in Lagrangian Hessian.............:      311

Total number of variables............................:       76
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

OptiSol(Opti {
  instance #1
  #variables: 1 (nx = 76)
  #parameters: 0 (np = 0)
  #constraints: 8 (ng = 27)
  CasADi solver allocated.
  CasADi solver was called: Solve_Succeeded
})

In [11]:
x_sol = integrate(x0, opti.value(dxs)).full()
q_sol = x_sol[: nq]
v_sol = x_sol[nq:]

In [12]:
viz.display(q_sol)

In [13]:
hasattr(viz.viewer, 'jupyter_cell') and viz.viewer.jupyter_cell()